# 06.2 - Knowledge Graph creation with LangChain

This notebook demonstrates how to automatically create a knowledge graph from research paper abstracts LangChain and LLMs.

## 01 - Setup

Ensure the necessary modules are installed and up to date.

In [1]:
%pip install --upgrade --quiet  langchain langchain-community langchain-experimental langchain-ollama json_repair

Note: you may need to restart the kernel to use updated packages.


Local LLMs are accessed via Ollama in this notebook. Ollama can be installed from https://ollama.com/.

## 02 - Data Preparation

For creating a knowledge graph with LangChain, we need to prepare the data by loading the research papers into LangChain Document objects.

In this step, we can either load the MIDAS papers that our LLM has classified as modeling papers (5700+ documents), or we can load the training papers from the MIDAS dataset (46). For starting, the latter, smaller dataset is recommended.

Load the classified papers (5700+ documents):

In [2]:
import pandas as pd
from genscai import paths
from langchain_core.documents import Document

df_modeling_papers = pd.read_json(paths.data / "modeling_papers_0.json", orient="records", lines=True)

documents = []

for row in df_modeling_papers.itertuples():
    documents.append(Document(id=row.id, page_content=row.abstract))

f"Papers loaded: {len(documents)}"

'Papers loaded: 5737'

Alternatively, load the training papers (46 documents):

In [3]:
import json
from genscai import paths
from langchain_core.documents import Document

with open(paths.data / "training_modeling_papers.json", "r") as f:
    data = json.load(f)

documents = []

for row in data:
    documents.append(Document(page_content=row["abstract"]))

f"Papers loaded: {len(documents)}"

'Papers loaded: 46'

## 03 - Create a Knowledge Graph

LangChain provides direct, experimental support for creating knowledge graphs from documents using LLMs. This is done by using the `LLMGraphTransformer` class.

Documentation on using LLMGraphTransformer can be found at:
https://python.langchain.com/docs/how_to/graph_constructing/

The prompt that LLMGraphTransformer uses to identify graph entities and relationships can be found at:
https://python.langchain.com/api_reference/_modules/langchain_experimental/graph_transformers/llm.html#create_unstructured_prompt

Inport the necessary modules so they can be used later:

In [4]:
from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain_ollama.llms import OllamaLLM

def print_graph_results(graph_documents: list[Document]) -> None:
    for doc in graph_documents:
        if len(doc.nodes) > 0:
            print(f"Paper ID: {doc.source.id}")
            print(f"Paper Abstract: {doc.source.page_content}")

            for node in doc.nodes:
                print(node)
                print(f"Node: {node.id}, Type: {node.type}")

            for rel in doc.relationships:
                print(f"Relationship: {rel.type}")
                print(f"   Source: {rel.source.id}, Type: {rel.source.type}")
                print(f"   Target: {rel.target.id}, Type: {rel.target.type}")

            print()

For the first test, allow the model to infer both node types and relationships. Here we'll use the Gemma 3 12B model.

In [5]:
llm = OllamaLLM(model="gemma3:12b", temperature=0.1)
transformer = LLMGraphTransformer(llm=llm)

transformer = LLMGraphTransformer(
    llm=llm,
)

# Process a single document for testing
graph_documents = transformer.convert_to_graph_documents(documents[:1])

print_graph_results(graph_documents)

Paper ID: None
Paper Abstract: Background: Since the appearance of the first case of COVID-19 in Morocco, the cumulative number of reported infectious cases continues to increase and, consequently, the government imposed the containment measure within the country. Our aim is to predict the impact of the compulsory containment on COVID-19 spread. Earlier knowledge of the epidemic characteristics of COVID-19 transmission related to Morocco will be of great interest to establish an optimal plan-of-action to control the epidemic.

Method: Using a Susceptible-Asymptomatic-Infectious model and the data of reported cumulative confirmed cases in Morocco from March 2nd to April 9, 2020, we determined the basic and control reproduction numbers and we estimated the model parameter values. Furthermore, simulations of different scenarios of containment are performed.

Results: Epidemic characteristics are predicted according to different rates of containment. The basic reproduction number is estima

Next, create a knowledge graph with specific node types.

In [6]:
transformer = LLMGraphTransformer(
    llm=llm,
    allowed_nodes=[
        "Disease Modeling Goal",
        "Diesase Modeling Technique",
        "Disease Model Data Requirement",
        "Disease Modeled",
        "Geographic Location",
    ],
)

# Process a single document for testing
graph_documents = transformer.convert_to_graph_documents(documents[:1])

print_graph_results(graph_documents)

Paper ID: None
Paper Abstract: Background: Since the appearance of the first case of COVID-19 in Morocco, the cumulative number of reported infectious cases continues to increase and, consequently, the government imposed the containment measure within the country. Our aim is to predict the impact of the compulsory containment on COVID-19 spread. Earlier knowledge of the epidemic characteristics of COVID-19 transmission related to Morocco will be of great interest to establish an optimal plan-of-action to control the epidemic.

Method: Using a Susceptible-Asymptomatic-Infectious model and the data of reported cumulative confirmed cases in Morocco from March 2nd to April 9, 2020, we determined the basic and control reproduction numbers and we estimated the model parameter values. Furthermore, simulations of different scenarios of containment are performed.

Results: Epidemic characteristics are predicted according to different rates of containment. The basic reproduction number is estima

Different models will produce varying results. The Mistral Small 3.1 model should produce better results than the Gemma 3 model; howver, the model requires more memory (~15GB at 4-bit quantization).

In [7]:
llm = OllamaLLM(model="mistral-small3.1:24b", temperature=0.15)
transformer = LLMGraphTransformer(llm=llm)

transformer = LLMGraphTransformer(
    llm=llm,
    allowed_nodes=[
        "Disease Modeling Goal",
        "Diesase Modeling Technique",
        "Disease Model Data Requirement",
        "Disease Modeled",
        "Geographic Location",
    ],
)

# Process a subset of the documents as a test
graph_documents = transformer.convert_to_graph_documents(documents[:1])

print_graph_results(graph_documents)

Paper ID: None
Paper Abstract: Background: Since the appearance of the first case of COVID-19 in Morocco, the cumulative number of reported infectious cases continues to increase and, consequently, the government imposed the containment measure within the country. Our aim is to predict the impact of the compulsory containment on COVID-19 spread. Earlier knowledge of the epidemic characteristics of COVID-19 transmission related to Morocco will be of great interest to establish an optimal plan-of-action to control the epidemic.

Method: Using a Susceptible-Asymptomatic-Infectious model and the data of reported cumulative confirmed cases in Morocco from March 2nd to April 9, 2020, we determined the basic and control reproduction numbers and we estimated the model parameter values. Furthermore, simulations of different scenarios of containment are performed.

Results: Epidemic characteristics are predicted according to different rates of containment. The basic reproduction number is estima

Relationships can be constrained as well. Either the names of the relatioships can be specificed, and the model will infter the nodes that are connected by these relationships, or allowed subject-predicate-object tuples can be specified.

In [8]:
allowed_relationships = [
    ("Disease Modeled", "LOCATION", "Geographic Location"),
]

transformer = LLMGraphTransformer(
    llm=llm,
    allowed_nodes=[
        "Disease Modeling Goal",
        "Diesase Modeling Technique",
        "Disease Model Data Requirement",
        "Disease Modeled",
        "Geographic Location",
    ],
    allowed_relationships=allowed_relationships,
)

# Process a subset of the documents as a test
graph_documents = transformer.convert_to_graph_documents(documents[:1])

print_graph_results(graph_documents)

Paper ID: None
Paper Abstract: Background: Since the appearance of the first case of COVID-19 in Morocco, the cumulative number of reported infectious cases continues to increase and, consequently, the government imposed the containment measure within the country. Our aim is to predict the impact of the compulsory containment on COVID-19 spread. Earlier knowledge of the epidemic characteristics of COVID-19 transmission related to Morocco will be of great interest to establish an optimal plan-of-action to control the epidemic.

Method: Using a Susceptible-Asymptomatic-Infectious model and the data of reported cumulative confirmed cases in Morocco from March 2nd to April 9, 2020, we determined the basic and control reproduction numbers and we estimated the model parameter values. Furthermore, simulations of different scenarios of containment are performed.

Results: Epidemic characteristics are predicted according to different rates of containment. The basic reproduction number is estima

Node and relationship properties can also be extracted from the data. These can either be specified as a list of properties, or the model can be allowed to infer them.

For infering node/relationship properties, LLMGraphTransformer requires that the model supports tool usage. Tool usage is only available using LangChain chat models (e.g. ChatOllama), and only a subset of models support tool usage. Inferring properties takes additionl time, since the process requires multiple LLM calls.

Unfortunately, the following code does not identify any graph elements, despite different Ollama models being used, that support tool usage.

In [9]:
from langchain_ollama import ChatOllama

llm = ChatOllama(model="mistral-small3.1:24b", temperature=0.15)

transformer = LLMGraphTransformer(
    llm=llm,
    allowed_nodes=[
        "Disease Modeling Goal",
        "Diesase Modeling Technique",
        "Disease Model Data Requirement",
        "Disease Modeled",
        "Geographic Location",
    ],
    allowed_relationships=[],
    # node_properties=True,
    # relationship_properties=True,
)

# Process a subset of the documents as a test
graph_documents = transformer.convert_to_graph_documents(documents[:1])

print_graph_results(graph_documents)

Alternatively, local Hugging Face models can be used to create knowledge graphs. However, there are still errors when including the node_properties or relationship_properties parameters.

In [10]:
%pip install --quiet --upgrade langchain-huggingface

Note: you may need to restart the kernel to use updated packages.


In [11]:
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint

llm = HuggingFaceEndpoint(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    task="conversational",
    max_new_tokens=512,
    do_sample=True,
)

chat_model = ChatHuggingFace(llm=llm)

transformer = LLMGraphTransformer(
    llm=chat_model,
    allowed_nodes=[
        "Disease Modeling Goal",
        "Diesase Modeling Technique",
        "Disease Model Data Requirement",
        "Disease Modeled",
        "Geographic Location",
    ],
    allowed_relationships=[],
    # node_properties=True,
    # relationship_properties=True,
)

# Process a subset of the documents as a test
graph_documents = transformer.convert_to_graph_documents(documents[:1])

print_graph_results(graph_documents)

Paper ID: None
Paper Abstract: Background: Since the appearance of the first case of COVID-19 in Morocco, the cumulative number of reported infectious cases continues to increase and, consequently, the government imposed the containment measure within the country. Our aim is to predict the impact of the compulsory containment on COVID-19 spread. Earlier knowledge of the epidemic characteristics of COVID-19 transmission related to Morocco will be of great interest to establish an optimal plan-of-action to control the epidemic.

Method: Using a Susceptible-Asymptomatic-Infectious model and the data of reported cumulative confirmed cases in Morocco from March 2nd to April 9, 2020, we determined the basic and control reproduction numbers and we estimated the model parameter values. Furthermore, simulations of different scenarios of containment are performed.

Results: Epidemic characteristics are predicted according to different rates of containment. The basic reproduction number is estima